In [1]:
import numpy as np
import pandas as pd

In [22]:
def choice_home(home_number):
    if home_number==1:
        return ["HHHHHHHHHHHHHHH",
                "HFFFFFFHFFHHHFH",
                "HFFHHFFHFFFFHFH",
                "HFFHHFFHFFFFHFH",
                "HFFFFFFHFFFFFFH",
                "HHHHHHFFFHHHHHH",
                "HFFFFHFFFHFFFFH",
                "HFFFFFFFFHFHHFH",
                "HFFFFHFFFFFFHFH",
                "HHHHHHFFFHFFFFH",
                "HFFFFHFFFHHHHHH",
                "HFFFFFFFFHFFFFH",
                "HHHHHHFFFHFFHFH",
                "HSFFFFFFFFFHHFH",
                "HHHHHHHHHHHHHHH"]
    if home_number==2:
        return ["HHHHHHHHHHHHHHHHHHHH",
                "HFFFFFHFFFFFHHHHFFFH",
                "HFFHFFFFFHFFFFHHHHFH",
                "HFFHFFFFFHFFFFFFHHFH",
                "HFFFFFHFHFFFFFFFHHFH",
                "HFFFFFHFHFFFFFHHFFFH",
                "HFFFFFFFHFHFFFFHHFFH",
                "HFFFFHFFFFFFHFFFHFFH",
                "HFFFHFFFFFHHFFFHFFFH",
                "HFFFFHHHFFFFFHHFFFFH",
                "HFFFFFHHHFFHHHFFFFFH",
                "HFFFFFHHFFFFFHFHFFFH",
                "HFFFFHHHHSFHFFFFHFFH",
                "HFFFFFHFFHFHFFHFFFFH",
                "HFFFFFHFFFFFHFFFHFFH",
                "HFFFFFHFHFFFFFHFHFFH",
                "HFFFFFHHFFFFHFFFFHFH",
                "HFFFFHHHFFFFHFFHFFFH",
                "HFFFHHFFFFFHHFFHFFFH",
                "HHHHHHHHHHHHHHHHHHHH"]

In [23]:
class Robot:
    np.random.seed(2)
    def __init__(self, home):
        self.gamma=0.5 #feedback oranı
        self.alpha=0.1 #öğrenme oranı
        self.epsilon=0.3 #
        self.max_episodes=100
        self.actions=['left','right','up','down']
        self.home = home
        self.y, self.x = self.create_robot_loc()
        self.map=self.create_map()
        self.update_map([self.y,self.x])
        self.nodes=[]
        
    def update_map(self,locs):
        self.map[locs[0],locs[1]]=1
    def create_map(self):
        return np.zeros([len(home),len(home[0])],int)
    def create_robot_loc(self):
        x=0
        y=0
        for line in home:
            for state in line:
                if (state == 'S'):
                    return y,x
                    break
                x+=1
            x=0
            y+=1
    def show_map(self):
        print(self.map)

    def get_loc(self):
        return self.y,self.x

    
    def set_loc(self,y,x):
        self.x=x
        self.y=y

    
    def s_acts(self):
        y,x=self.get_loc()
        acts=[]
        if home[y][x+1]!='H':
            if self.map[y][x+1]==0:
                acts.append('right')
        if home[y][x-1]!='H':
            if self.map[y][x-1]==0:
                acts.append('left')
        if home[y-1][x]!='H':
            if self.map[y-1][x]==0:
                acts.append('up')
        if home[y+1][x]!='H':
            if self.map[y+1][x]==0:
                acts.append('down')
        return acts

    
    def choise_act(self,Table,acts):
        state_acts=Table.loc[self.get_loc()]
        if (np.random.uniform() > self.epsilon) or (state_acts[acts]==0).all():
            act_name = np.random.choice(acts)
        else:
            act_name=state_acts[acts].idxmax()
        return act_name

    
    def qTable(self,states):
        q_table = pd.DataFrame(np.zeros((len(states), len(self.actions)))
                               ,index=pd.MultiIndex.from_tuples(states, names=['Row', 'Column']),columns=self.actions)
        return q_table



    def node_process(self,s_acts):
        if len(s_acts)>1:
            self.nodes.append(self.get_loc())         
        elif  len(s_acts)==0:
            if self.get_loc() in self.nodes:
                self.nodes.remove(self.get_loc())
                if len(self.nodes)==0:
                    self.set_loc(self.create_robot_loc()[0],self.create_robot_loc()[1])
                else:
                    node=self.nodes[len(self.nodes)-1]
                    self.set_loc(node[0],node[1])
            else:
                node=self.nodes[len(self.nodes)-1]
                self.set_loc(node[0],node[1])
        
    def act_feedback(self,act):
        y,x=self.get_loc()
        if act=='right':
            if self.map[y][x+1]==0:
                self.map[y][x+1]=1
                self.set_loc(y,x+1)
                R=1
        if act=='left':
            if self.map[y][x-1]==0:
                self.map[y][x-1]=1
                self.set_loc(y,x-1)
                R=1
        if act=='up':
             if self.map[y-1][x]==0:
                self.map[y-1][x]=1
                self.set_loc(y-1,x)
                R=1
        if act=='down':
            if self.map[y+1][x]==0:
                self.map[y+1][x]=1
                self.set_loc(y+1,x)
                R=1
        return R   
    
    def learn(self):
        table=self.qTable(states)
        for ep in range(self.max_episodes):
            self.map=self.create_map()
            self.update_map(self.create_robot_loc())
            self.set_loc(self.create_robot_loc()[0],self.create_robot_loc()[1])
            self.home=home
            Done=True
            self.nodes=[]
            while Done==True:
                State=self.get_loc()
                s_acts = self.s_acts()
                self.node_process(s_acts)
                if not s_acts:
                    if self.get_loc()==self.create_robot_loc():
                        Done=False
                    continue
                else:
                    act=self.choise_act(table,s_acts)
                    R = self.act_feedback(act)
                    target = R+self.gamma*table.loc[State].max()
                    predict=table.loc[State,act]
                    table.loc[State,act]+=self.alpha*(target-predict)
        self.show_map()
        return table

In [24]:
home=choice_home(2)
robot1=Robot(home)
states = [(i, j) for i in range(len(home)) for j in range(len(home[0]))]

In [25]:
table=robot1.learn()

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 1 1 1 0 1 1 1 1 1 0 0 0 0 1 1 1 0]
 [0 1 1 0 1 1 1 1 1 0 1 1 1 1 0 0 0 0 1 0]
 [0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 0]
 [0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 0 1 0]
 [0 1 1 1 1 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0]
 [0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 0 1 1 0]
 [0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1 0]
 [0 1 1 1 0 1 1 1 1 1 0 0 1 1 1 0 1 1 1 0]
 [0 1 1 1 1 0 0 0 1 1 1 1 1 0 0 1 1 1 1 0]
 [0 1 1 1 1 1 0 0 0 1 1 0 0 0 1 1 1 1 1 0]
 [0 1 1 1 1 1 0 0 1 1 1 1 1 0 1 0 1 1 1 0]
 [0 1 1 1 1 0 0 0 0 1 1 0 1 1 1 1 0 1 1 0]
 [0 1 1 1 1 1 0 1 1 0 1 0 1 1 0 1 1 1 1 0]
 [0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 0]
 [0 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 0 1 1 0]
 [0 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 0 1 0]
 [0 1 1 1 1 0 0 0 1 1 1 1 0 1 1 0 1 1 1 0]
 [0 1 1 1 0 0 1 1 1 1 1 0 0 1 1 0 1 1 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
